In [11]:
from bs4 import BeautifulSoup
import requests
import json


with open('assets/lesson_topics.json', 'r', encoding='utf-8') as f:
    lessonTopics = json.load(f)

questions_collection = []

with open('assets/topics_questions.json', 'r', encoding='utf-8') as f:
    questions_data = json.load(f)

for url, topics_map in lessonTopics.items():
    lesson_id = url.split('/').pop().split('?')[0]
    categoryId = url.split('/').pop().split('?s=')[1].split('&')[0]
    category = int(categoryId) + 3
    for topicName, subTopicList in topics_map.items():
        for subTopic in subTopicList:
            subTopicId = subTopic['value']
            subTopicName = subTopic['text']
            indexOfTopicInTopicsMap = list(topics_map.keys()).index(topicName)
            for questionUrl, questionData in questions_data.items():
                if questionUrl == (f"https://ogmmateryal.eba.gov.tr/api/soru-tur-listele?id={subTopicId}"):
                    for question in questionData['questions']:
                        questionDoc = {
                            "topicPath": f"category/{category}/lessons/{lesson_id}/topics/{indexOfTopicInTopicsMap}/subtopics/{subTopicId}",
                            "id": question['id'],
                            "answer": question['cevap'],
                            "url": f"https://ogmmateryal.eba.gov.tr/soru-bankasi/{lesson_id}/test?s={categoryId}&k={subTopicId}&id={question['id']}&t=css&ks=0&os=0&zs=0"
                        }

                        questions_collection.append(questionDoc)


In [29]:
import time
firebaseQuestions = []


# Create and open processed questions file


for url, topics_map in lessonTopics.items():
    
    lesson_id = url.split('/').pop().split('?')[0]
    categoryId = url.split('/').pop().split('?s=')[1].split('&')[0]
    # Create file name using lesson_id and categoryId
    file_name = f"{lesson_id}_{categoryId}_questions.json"
    
    # Initialize empty list for questions
    questionsAsDict = []
    
    # Open file in write mode
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(questionsAsDict, f, ensure_ascii=False)
    for question in questions_collection:
        if question['url'].startswith(f"https://ogmmateryal.eba.gov.tr/soru-bankasi/{lesson_id}/test?s={categoryId}"):
            # Print progress
            print(f"{questions_collection.index(question) + 1}/{len(questions_collection)}", end='\r')
            # Get question HTML from URL with retry logic and delay
            max_retries = 3
            retry_delay = 2  # seconds

            for attempt in range(max_retries):
                try:
                    response = requests.get(question['url'])
                    if response.status_code == 200:
                        break
                    elif response.status_code == 429:  # Too Many Requests
                        if attempt < max_retries - 1:  # Don't sleep on last attempt
                            time.sleep(retry_delay)
                            retry_delay *= 2  # Exponential backoff
                        continue
                    else:
                        response.raise_for_status()
                except requests.exceptions.RequestException as e:
                    if attempt < max_retries - 1:
                        time.sleep(retry_delay)
                        retry_delay *= 2
                        continue
                    print(
                        f"Failed to get question {question['id']} after {max_retries} attempts: {e}")
                    continue

            soup = BeautifulSoup(response.text, 'html.parser')
            question_html = soup.find('div', class_='question-item')
            if question_html:
                question["questionAsHtml"] = str(question_html)
                texts = question_html.stripped_strings
                question_text = ' '.join(texts)
                question["questionText"] = question_text

                if question["questionText"].startswith(f"Soru 1: {question['id']}"):
                    question["questionText"] = question["questionText"].split(
                        f"Soru 1: {question['id']}")[-1].strip()

                firebaseQuestions.append(question)
                questionsAsDict.append(question)
                with open(file_name, 'w', encoding='utf-8') as f: 
                    json.dump(questionsAsDict, f, ensure_ascii=False)

            # Add delay between requests to avoid rate limiting
            time.sleep(1)

    len(firebaseQuestions)







In [14]:
filePaths = [
    'biyoloji_6_questions.json',
    'biyoloji_7_questions.json',
    'biyoloji_8_questions.json', 
    'biyoloji_9_questions.json',
    'cografya_6_questions.json',
    'cografya_7_questions.json',
    'cografya_8_questions.json',
    'cografya_9_questions.json',
    'dikab_6_questions.json',
    'dikab_7_questions.json',
    'dikab_8_questions.json',
    'dikab_9_questions.json',
    'felsefe_7_questions.json',
    'felsefe_8_questions.json',
    'fizik_6_questions.json',
    'fizik_7_questions.json', 
    'fizik_8_questions.json',
    'fizik_9_questions.json',
    'fl-biyoloji_6_questions.json',
    'fl-biyoloji_7_questions.json',
    'fl-biyoloji_8_questions.json',
    'fl-biyoloji_9_questions.json',
    'fl-fizik_6_questions.json',
    'fl-fizik_7_questions.json',
    'fl-fizik_8_questions.json',
    'fl-fizik_9_questions.json',
    'fl-kimya_6_questions.json',
    'fl-kimya_7_questions.json',
    'fl-kimya_8_questions.json',
    'fl-kimya_9_questions.json',
    'fl-matematik_6_questions.json',
    'fl-matematik_7_questions.json',
    'fl-matematik_8_questions.json',
    'fl-matematik_9_questions.json',
    'ingilizce_6_questions.json',
    'ingilizce_7_questions.json',
    'ingilizce_8_questions.json',
    'ingilizce_9_questions.json',
    'inkilap-tarihi_9_questions.json',
    'kimya_6_questions.json',
    'kimya_7_questions.json',
    'kimya_8_questions.json',
    'kimya_9_questions.json',
    'matematik_6_questions.json',
    'matematik_7_questions.json',
    'matematik_8_questions.json',
    'matematik_9_questions.json',
    'tarih_6_questions.json',
    'tarih_7_questions.json',
    'tarih_8_questions.json',
    'tde_6_questions.json',
    'tde_7_questions.json',
    'tde_8_questions.json',
    'tde_9_questions.json',
]

# import json
# stats = {}

# for filename in filePaths:
#     with open("questions/" + filename, 'r', encoding='utf-8') as f: 
#         questions = json.load(f)
        
#         # Initialize counters
#         image_count = 0
#         no_image_count = 0
#         total_questions = len(questions)
        
#         for question in questions:
#             # Check for href tags and add withImage flag
#             has_image = 'questionAsHtml' in question and 'img' in question['questionAsHtml'].lower()
#             question['withImage'] = has_image
            
#             if has_image:
#                 image_count += 1
#             else:
#                 no_image_count += 1
                
#         # Save the modified questions back to file
#         with open(filename, 'w', encoding='utf-8') as f:
#             json.dump(questions, f, ensure_ascii=False, indent=2)
                
#         # Add stats to dict if there are any href tags
#         if image_count > 0:
#             stats[filename] = {
#                 'total_questions': total_questions,
#                 'image_count': image_count,
#                 'no_image_count': no_image_count
#             }

# print("File statistics:")
# for file, file_stats in stats.items():
#     print(f"\n{file}:")
#     print(f"  Total questions: {file_stats['total_questions']}")
#     print(f"  Questions with images: {file_stats['image_count']}")
#     print(f"  Questions without images: {file_stats['no_image_count']}")


# # Calculate total questions with no images across all files
# total_no_image = sum(file_stats['no_image_count'] for file_stats in stats.values())
# print(f"\nTotal questions without images across all files: {total_no_image}")


In [17]:


import firebase_admin
from firebase_admin import credentials, firestore
from tqdm import tqdm
from google.cloud import firestore
from google.oauth2 import service_account
import json
import os
# Initialize credentials from service account
creds = service_account.Credentials.from_service_account_file('firebase_credentials.json')

# Initialize Firestore client with service account
db = firestore.Client(credentials=creds)

# Create a batch writer
def batch_write(db, docs, batch_size=500):
    batch = db.batch()
    count = 0
    total_docs = len(docs)
    
    for i, doc in enumerate(tqdm(docs)):
        # Create document reference with question ID
        doc_ref = db.collection('questions').document(str(doc['id']))
        batch.set(doc_ref, doc)
        count += 1
        
        # Print progress
        print(f"Processing question {i+1}/{total_docs}")
        
        # When batch_size is reached, commit and reset
        if count == batch_size:
            batch.commit()
            batch = db.batch()
            count = 0
    
    # Commit any remaining documents
    if count > 0:
        batch.commit()




FileNotFoundError: [Errno 2] No such file or directory: 'firebase_credentials.json'

In [16]:
# Read all question files and combine questions
all_questions = []
for filename in filePaths:
    with open("questions/" + filename, 'r', encoding='utf-8') as f:
        questions = json.load(f)
        all_questions.extend(questions)

# Write all questions to Firestore in batches
batch_write(db, all_questions)
print(f"Successfully wrote {len(all_questions)} questions to Firestore")

  0%|          | 0/36940 [00:00<?, ?it/s]

Processing question 1/36940
Processing question 2/36940
Processing question 3/36940
Processing question 4/36940
Processing question 5/36940
Processing question 6/36940
Processing question 7/36940
Processing question 8/36940
Processing question 9/36940
Processing question 10/36940
Processing question 11/36940
Processing question 12/36940
Processing question 13/36940
Processing question 14/36940
Processing question 15/36940
Processing question 16/36940
Processing question 17/36940
Processing question 18/36940
Processing question 19/36940
Processing question 20/36940
Processing question 21/36940
Processing question 22/36940
Processing question 23/36940
Processing question 24/36940
Processing question 25/36940
Processing question 26/36940
Processing question 27/36940
Processing question 28/36940
Processing question 29/36940
Processing question 30/36940
Processing question 31/36940
Processing question 32/36940
Processing question 33/36940
Processing question 34/36940
Processing question 35/

  1%|▏         | 500/36940 [00:02<03:18, 183.77it/s]

Processing question 501/36940
Processing question 502/36940
Processing question 503/36940
Processing question 504/36940
Processing question 505/36940
Processing question 506/36940
Processing question 507/36940
Processing question 508/36940
Processing question 509/36940
Processing question 510/36940
Processing question 511/36940
Processing question 512/36940
Processing question 513/36940
Processing question 514/36940
Processing question 515/36940
Processing question 516/36940
Processing question 517/36940
Processing question 518/36940
Processing question 519/36940
Processing question 520/36940
Processing question 521/36940
Processing question 522/36940
Processing question 523/36940
Processing question 524/36940
Processing question 525/36940
Processing question 526/36940
Processing question 527/36940
Processing question 528/36940
Processing question 529/36940
Processing question 530/36940
Processing question 531/36940
Processing question 532/36940
Processing question 533/36940
Processing

  3%|▎         | 1000/36940 [00:04<02:47, 213.94it/s]

...

Processing question 35501/36940
Processing question 35502/36940
Processing question 35503/36940
Processing question 35504/36940
Processing question 35505/36940
Processing question 35506/36940
Processing question 35507/36940
Processing question 35508/36940
Processing question 35509/36940
Processing question 35510/36940
Processing question 35511/36940
Processing question 35512/36940
Processing question 35513/36940
Processing question 35514/36940
Processing question 35515/36940
Processing question 35516/36940
Processing question 35517/36940
Processing question 35518/36940
Processing question 35519/36940
Processing question 35520/36940
Processing question 35521/36940
Processing question 35522/36940
Processing question 35523/36940
Processing question 35524/36940
Processing question 35525/36940
Processing question 35526/36940
Processing question 35527/36940
Processing question 35528/36940
Processing question 35529/36940
Processing question 35530/36940
Processing question 35531/36940
Processi

 97%|█████████▋| 36000/36940 [02:59<00:07, 132.86it/s]

Processing question 36001/36940
Processing question 36002/36940
Processing question 36003/36940
Processing question 36004/36940
Processing question 36005/36940
Processing question 36006/36940
Processing question 36007/36940
Processing question 36008/36940
Processing question 36009/36940
Processing question 36010/36940
Processing question 36011/36940
Processing question 36012/36940
Processing question 36013/36940
Processing question 36014/36940
Processing question 36015/36940
Processing question 36016/36940
Processing question 36017/36940
Processing question 36018/36940
Processing question 36019/36940
Processing question 36020/36940
Processing question 36021/36940
Processing question 36022/36940
Processing question 36023/36940
Processing question 36024/36940
Processing question 36025/36940
Processing question 36026/36940
Processing question 36027/36940
Processing question 36028/36940
Processing question 36029/36940
Processing question 36030/36940
Processing question 36031/36940
Processi

100%|██████████| 36940/36940 [03:01<00:00, 203.35it/s]


Processing question 36501/36940
Processing question 36502/36940
Processing question 36503/36940
Processing question 36504/36940
Processing question 36505/36940
Processing question 36506/36940
Processing question 36507/36940
Processing question 36508/36940
Processing question 36509/36940
Processing question 36510/36940
Processing question 36511/36940
Processing question 36512/36940
Processing question 36513/36940
Processing question 36514/36940
Processing question 36515/36940
Processing question 36516/36940
Processing question 36517/36940
Processing question 36518/36940
Processing question 36519/36940
Processing question 36520/36940
Processing question 36521/36940
Processing question 36522/36940
Processing question 36523/36940
Processing question 36524/36940
Processing question 36525/36940
Processing question 36526/36940
Processing question 36527/36940
Processing question 36528/36940
Processing question 36529/36940
Processing question 36530/36940
Processing question 36531/36940
Processi